### Projeto Ciência de Dados - Titanic

- COLOCAR OS OBJETIVOS DO PROJETO

##### Importar as Bibliotecas

In [1]:
import pandas as pd
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

##### Importar a Base de Treino

In [2]:
treino = pd.read_csv('train.csv')
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


##### Importar a Base de Teste

In [3]:
# Visualizando a base de teste
teste = pd.read_csv('test.csv')
teste.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


##### Tratar as Bases

In [4]:
# Calcular a Cardinalidade das Colunas
cardinalidades = treino.nunique().sort_values(ascending=False)
cardinalidades = pd.DataFrame({'Coluna': cardinalidades.index, 'Cardinalidade': cardinalidades.values})
cardinalidades

,Coluna,Cardinalidade
0,PassengerId,891
1,Name,891
2,Ticket,681
3,Fare,248
4,Cabin,147
5,Age,88
6,SibSp,7
7,Parch,7
8,Pclass,3
9,Embarked,3


In [5]:
# Eliminar as Colunas com Elevada Cardinalidade
treino = treino.drop(['Name', 'Ticket', 'Cabin'], axis=1)
teste = teste.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [6]:
# Usar a Média para Substituir Valores Nulos na Coluna de Idade
treino.loc[treino['Age'].isnull(), 'Age'] = treino.Age.mean()
teste.loc[teste['Age'].isnull(), 'Age'] = teste.Age.mean()

In [7]:
# Tratar a Coluna Embarked da Base de Treino Usando a Moda 
treino.loc[treino['Embarked'].isnull(), 'Embarked'] = treino['Embarked'].mode()[0]

In [8]:
# Tratar a Coluna Fare da Base de Teste Usando a Média
teste.loc[teste['Fare'].isnull(), 'Fare'] = teste['Fare'].mean()

##### Tratar as Colunas - Engenharia dos Dados

In [9]:
# Tratar a Coluna de Sexo Usando 1 para Masculino e 0 para Feminino
treino['MaleCheck'] = treino['Sex'].apply(lambda x: 1 if x == 'male' else 0)
teste['MaleCheck'] = teste['Sex'].apply(lambda x: 1 if x == 'male' else 0)

In [10]:
# Tratar as Colunas Age e Fare (Estão em Escalas Diferentes das Demais) usando o RobustScaler
transformer = RobustScaler().fit(treino[['Age', 'Fare']])
treino[['Age', 'Fare']] = transformer.transform(treino[['Age', 'Fare']])

transformer = RobustScaler().fit(teste[['Age', 'Fare']])
teste[['Age', 'Fare']] = transformer.transform(teste[['Age', 'Fare']])

In [11]:
# Adicionar a Coluna Sozinho
def sozinho(a, b):
    if (a == 0 and b == 0):
        return 1
    else:
        return 0
    
treino['Sozinho'] = treino.apply(lambda x: sozinho(x['SibSp'], x['Parch']), axis=1)
teste['Sozinho'] = teste.apply(lambda x: sozinho(x['SibSp'], x['Parch']), axis=1)

In [12]:
# Criar a Columa Familiares
treino['Familiares'] = treino['SibSp'] + treino['Parch']
teste['Familiares'] = treino['SibSp'] + treino['Parch']

In [13]:
# Tratar a Coluna Embarked com o OrdinalEncoder
categorias = ['S', 'C', 'Q']

enc = OrdinalEncoder(categories=[categorias], dtype='int32')
enc = enc.fit(treino[['Embarked']])

treino['Embarked'] = enc.transform(treino[['Embarked']])
teste['Embarked'] = enc.transform(teste[['Embarked']])

In [14]:
# Apagar a Coluna de Sexo
treino = treino.drop('Sex', axis=1)
teste = teste.drop('Sex', axis=1)

##### Visualizar a Base de Treino Tratada

In [15]:
treino.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Sozinho,Familiares
0,1,0,3,-0.592240,1,0,-0.312011,0,1,0,1
1,2,1,1,0.638529,1,0,2.461242,1,0,0,1
2,3,1,3,-0.284548,0,0,-0.282777,0,0,1,0
3,4,1,1,0.407760,1,0,1.673732,0,0,0,1
4,5,0,3,0.407760,0,0,-0.277363,0,1,1,0


##### Utilizar Modelos de Classificação
- Serão Utilizados os seguintes Modelos de Classificação: **LogisticRegression**, **RandomForestClassifier**, **MLPClassifier**, **DecisionTreeClassifier** e **KNeighborsClassifier**
- Primeiramente, serão Feitas as **Previsões** e, logo após, as suas **Avaliações**, utilizando métricas como **Acurácia**, **Precisão**, **Recall**, **F1-score** e **Matriz de Confusão**
- Os Valores Passados nos **Parâmetros dos Modelos** (tirando o random_state) foram previamente testados e selecionados por resultarem em um **Melhor Score**

##### FAZER UMA PARTE SEM OS PARÂMETROS E OUTRA COM OS MELHORES PARÂMETROS

In [16]:
# Ignorando os avisos
warnings.filterwarnings('ignore')

# Separar a Base de Treino em x e y
x = treino.drop(['PassengerId', 'Survived'], axis=1)
y = treino['Survived']

# Separar em Dados de Treino e Dados de Teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

In [17]:
# Criar os Modelos de Classificação Sem os Melhores Parâmetros
models = [
    LogisticRegression(random_state=10),
    RandomForestClassifier(random_state=10),
    MLPClassifier(random_state=10),
    DecisionTreeClassifier(random_state=10),
    KNeighborsClassifier(n_neighbors=3)
]

# Percorrer cada um dos Modelos
for model in models:
    # Fazer as Previsões
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    # Calcular a Acurácia
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Acurácia do Modelo {type(model).__name__}: {accuracy:.2f}")

    # Calcular a Precisão
    precision = precision_score(y_test, y_pred)
    print(f"Precisão do Modelo {type(model).__name__}: {precision:.2f}")

    # Calcular o Recall
    recall = recall_score(y_test, y_pred)
    print(f"Recall do Modelo {type(model).__name__}: {recall:.2f}")

    # Calcular a F1-score
    f1 = f1_score(y_test, y_pred)
    print(f"F1-score do Modelo {type(model).__name__}: {f1:.2f}")

    # Matriz de Confusão
    cm = confusion_matrix(y_test, y_pred)
    print("Matriz de Confusão:")
    print(cm)
    print("---------------------------")

Acurácia do Modelo LogisticRegression: 0.83
Precisão do Modelo LogisticRegression: 0.76
Recall do Modelo LogisticRegression: 0.76
F1-score do Modelo LogisticRegression: 0.76
Matriz de Confusão:
[[102  15]
 [ 15  47]]
---------------------------
Acurácia do Modelo RandomForestClassifier: 0.82
Precisão do Modelo RandomForestClassifier: 0.72
Recall do Modelo RandomForestClassifier: 0.76
F1-score do Modelo RandomForestClassifier: 0.74
Matriz de Confusão:
[[99 18]
 [15 47]]
---------------------------
Acurácia do Modelo MLPClassifier: 0.83
Precisão do Modelo MLPClassifier: 0.79
Recall do Modelo MLPClassifier: 0.68
F1-score do Modelo MLPClassifier: 0.73
Matriz de Confusão:
[[106  11]
 [ 20  42]]
---------------------------
Acurácia do Modelo DecisionTreeClassifier: 0.74
Precisão do Modelo DecisionTreeClassifier: 0.61
Recall do Modelo DecisionTreeClassifier: 0.69
F1-score do Modelo DecisionTreeClassifier: 0.65
Matriz de Confusão:
[[89 28]
 [19 43]]
---------------------------
Acurácia do Mode

In [18]:
# Criar os Modelos de Classificação Testando os Parâmetros
models = {
    'LogisticRegression': (LogisticRegression(), {'penalty': ['l1', 'l2'], 'C': [0.01, 0.1, 1, 10], 
                                                  'solver': ['lbfgs', 'liblinear', 'saga'], 
                                                  'max_iter': [100, 1000, 5000, 10000]}),
    'RandomForestClassifier': (RandomForestClassifier(), {'n_estimators': [100, 200, 500, 1000], 
                                                          'criterion': ['gini', 'entropy', 'log_loss'], 
                                                          'max_depth': [2, 4, 6, 8, 10, None], 
                                                          'max_features': ['sqrt', 'log2', None],
                                                          'min_samples_split': [2, 5],
                                                          'min_samples_leaf': [1, 2]}),
    'MLPClassifier': (MLPClassifier(), {'hidden_layer_sizes': [(50,), (100,)], 
                                        'solver':  ['lbfgs', 'sgd', 'adam'], 
                                        'alpha': [10.0**(-1), 10.0**(-5), 10.0**(-7), 10.0**(-10)], 
                                        'max_iter': [200, 500, 1000, 5000],
                                        'activation': ['relu', 'tanh']}),
    'DecisionTreeClassifier': (DecisionTreeClassifier(), {'max_depth': [None, 10, 20],
                                                          'min_samples_split': [2, 5],
                                                          'min_samples_leaf': [1, 2]}),
    'KNeighborsClassifier': (KNeighborsClassifier(), {'n_neighbors': [3, 5, 7],
                                                      'weights': ['uniform', 'distance'],
                                                      'algorithm': ['auto', 'ball_tree']})
}

# Definir o K-Fold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Aplicar o GridSearchCV para cada modelo
for model_name, (model, param_grid) in models.items():
    grid_search = GridSearchCV(model, param_grid, scoring='accuracy', cv=kfold)
    grid_search.fit(x_train, y_train)
    
    print(f"Melhores Parâmetros para {model_name}: {grid_search.best_params_}")
    print(f"Acurácia do {model_name} como os Melhores Parâmetros: {grid_search.best_score_}")


Melhores parâmetros para LogisticRegression: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
Melhores parâmetros para RandomForestClassifier: {'criterion': 'log_loss', 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Melhores parâmetros para MLPClassifier: {'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': (50,), 'max_iter': 1000, 'solver': 'adam'}
Melhores parâmetros para DecisionTreeClassifier: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Melhores parâmetros para KNeighborsClassifier: {'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'uniform'}


In [31]:
# Criar os Modelos de Classificação Com os Melhores Parâmetros
models = [
    LogisticRegression(random_state=10, C=1, max_iter=100, penalty='l2', solver='lbfgs'),
    RandomForestClassifier(random_state=10, criterion='log_loss', max_depth=8, max_features=None,
                           min_samples_leaf=2, min_samples_split=5, n_estimators=200),
    MLPClassifier(random_state=10, activation='relu', alpha=0.1, 
                  hidden_layer_sizes=(50,), max_iter=100, solver='adam'),
    DecisionTreeClassifier(random_state=10, max_depth=10, min_samples_leaf=1, min_samples_split=2),
    KNeighborsClassifier(n_neighbors=3, algorithm='auto', weights='uniform')
]

# Percorrer cada um dos Modelos
for model in models:
    # Fazer as Previsões
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    # Calcular a Acurácia
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Acurácia do Modelo {type(model).__name__}: {accuracy:.2f}")

    # Calcular a Precisão
    precision = precision_score(y_test, y_pred)
    print(f"Precisão do Modelo {type(model).__name__}: {precision:.2f}")

    # Calcular o Recall
    recall = recall_score(y_test, y_pred)
    print(f"Recall do Modelo {type(model).__name__}: {recall:.2f}")

    # Calcular a F1-score
    f1 = f1_score(y_test, y_pred)
    print(f"F1-score do Modelo {type(model).__name__}: {f1:.2f}")

    # Matriz de Confusão
    cm = confusion_matrix(y_test, y_pred)
    print("Matriz de Confusão:")
    print(cm)
    print("---------------------------")

Acurácia do Modelo LogisticRegression: 0.83
Precisão do Modelo LogisticRegression: 0.76
Recall do Modelo LogisticRegression: 0.76
F1-score do Modelo LogisticRegression: 0.76
Matriz de Confusão:
[[102  15]
 [ 15  47]]
---------------------------
Acurácia do Modelo RandomForestClassifier: 0.85
Precisão do Modelo RandomForestClassifier: 0.81
Recall do Modelo RandomForestClassifier: 0.74
F1-score do Modelo RandomForestClassifier: 0.77
Matriz de Confusão:
[[106  11]
 [ 16  46]]
---------------------------
Acurácia do Modelo MLPClassifier: 0.83
Precisão do Modelo MLPClassifier: 0.77
Recall do Modelo MLPClassifier: 0.71
F1-score do Modelo MLPClassifier: 0.74
Matriz de Confusão:
[[104  13]
 [ 18  44]]
---------------------------
Acurácia do Modelo DecisionTreeClassifier: 0.78
Precisão do Modelo DecisionTreeClassifier: 0.68
Recall do Modelo DecisionTreeClassifier: 0.69
F1-score do Modelo DecisionTreeClassifier: 0.69
Matriz de Confusão:
[[97 20]
 [19 43]]
---------------------------
Acurácia do 

##### Fazer a Previsão para os Dados de Teste
- Será utilizado o Modelo com Melhor Previsão e Melhores Métricas, no caso o **RandomForestClassifier**

In [22]:
# Comparar a Base de Treino com a Base de Teste
x_train.head()

,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Sozinho,Familiares
57,3,-0.092240,0,0,-0.312911,1,1,1,0
717,2,-0.207624,0,0,-0.171255,0,0,1,0
431,3,0.000000,1,0,0.071279,0,0,0,1
633,1,0.000000,0,0,-0.626005,0,1,1,0
163,3,-0.976855,0,0,-0.250836,0,1,1,0


In [23]:
teste.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Sozinho,Familiares
0,892,3,0.331562,0,0,-0.280670,2,1,1,1
1,893,3,1.311954,1,0,-0.315800,0,0,0,1
2,894,2,2.488424,0,0,-0.201943,2,1,1,0
3,895,3,-0.256674,0,0,-0.245367,0,1,1,1
4,896,3,-0.648831,1,1,-0.091793,0,0,0,0


In [24]:
# Excluir a Coluna PassengerId, para que as Bases fiquem com as Mesmas Colunas
x_teste = teste.drop('PassengerId', axis=1)

In [25]:
# Fazer as Previsões na Base de Teste
y_pred = models[1].predict(x_teste)

In [26]:
# Criar uma Nova Coluna com as Previsões
teste['Survived'] = y_pred

In [27]:
# Selecionar apenas a Coluna de Id e Survived e Exportar
base_envio = teste[['PassengerId','Survived']]
base_envio.to_csv('resultados.csv', index=False)